In [1]:
!pip install openpyxl

In [2]:
#dependencies
import pandas as pd
import datetime
from sqlalchemy import create_engine
import glob

In [3]:
# U.S. Gasoline and Diesel Retail Prices 1995-2021
csv_file = "PET_PRI_GND_DCUS_NUS_W.csv"
us_gas_price_df = pd.read_csv(csv_file)
us_gas_price_df.head()

,Date,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


In [4]:
#renaming the columns
gas_prices_df = us_gas_price_df.rename(columns={"A1":"All Grades All Formulations",
                                                     "A2":"All Grades Conventional",
                                                     "A3":"All Grades Reformulated",
                                                     "R1":"Regular All Formulations",
                                                     "R2":"Regular Conventional", 
                                                     "R3":"Regular Reformulated",
                                                     "M1":"Midgrade All Formulations",
                                                     "M2":"Midgrade Conventional", 
                                                     "M3":"Midgrade Reformulated", 
                                                     "P1":"Premium All Formulations", 
                                                     "P2":"Premium Conventional", 
                                                     "P3":"Premium Reformulated", 
                                                     "D1":"No 2 Diesel"}) 
gas_prices_df

,Date,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,12/28/2020,2.330,2.225,2.535,2.243,2.158,2.423,2.634,2.482,2.858,2.889,2.770,3.031,2.635
1357,01/04/2021,2.336,2.227,2.549,2.249,2.160,2.437,2.639,2.484,2.867,2.895,2.771,3.042,2.640
1358,01/11/2021,2.403,2.298,2.610,2.317,2.232,2.498,2.702,2.550,2.927,2.959,2.839,3.101,2.670
1359,01/18/2021,2.464,2.351,2.688,2.379,2.285,2.579,2.759,2.601,2.995,3.014,2.885,3.166,2.696


In [5]:
#pull year out of Date column in us_gas_price df
gas_prices_year = pd.DatetimeIndex(gas_prices_df['Date']).year
#add column back to dataframe in preparation for join
gas_prices_df['Year'] = gas_prices_year
gas_prices_df.head()

,Date,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel,Year
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104,1995
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102,1995
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100,1995
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095,1995
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090,1995


In [6]:
#average all the gas prices by the year via groupby
avg_annual_gas = gas_prices_df.groupby(['Year']).mean()

avg_annual_gas

,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
Year,,,,,,,,,,,,,
1995,1.157712,1.143538,1.219615,1.110865,1.103308,1.162500,1.200635,1.196000,1.284904,1.297981,1.287154,1.368846,1.109154
1996,1.244528,1.230830,1.296849,1.199075,1.191925,1.241792,1.285170,1.280151,1.363962,1.380642,1.370679,1.438057,1.235283
1997,1.244250,1.227788,1.306808,1.199038,1.188692,1.252327,1.284192,1.276712,1.377269,1.380058,1.368481,1.446981,1.198308
1998,1.071712,1.056212,1.133135,1.029654,1.016654,1.077788,1.121904,1.105885,1.201058,1.213865,1.198788,1.275692,1.043885
1999,1.176058,1.155519,1.260154,1.135808,1.115981,1.195154,1.233596,1.207019,1.331115,1.319635,1.296558,1.399692,1.120519
2000,1.522731,1.500692,1.605135,1.483808,1.462308,1.542865,1.576462,1.548750,1.670308,1.662942,1.639385,1.741250,1.490769
2001,1.460302,1.422019,1.548849,1.420057,1.383830,1.498415,1.513283,1.467792,1.615226,1.602472,1.561396,1.691623,1.400509
2002,1.385962,1.352212,1.452885,1.344500,1.312673,1.408423,1.439423,1.399212,1.516654,1.530154,1.495731,1.593981,1.318750
2003,1.603019,1.554577,1.700635,1.561096,1.515808,1.654712,1.658231,1.600615,1.769558,1.747808,1.697365,1.841942,1.509327


In [7]:
#isolating the last 5 years of interest
last_5yrs = avg_annual_gas.loc[2017:]

last_5yrs

,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
Year,,,,,,,,,,,,,
2017,2.527885,2.447769,2.691231,2.414615,2.333173,2.585500,2.681615,2.595096,2.848885,2.916500,2.864577,3.013019,2.650038
2018,2.813189,2.714358,3.008151,2.718547,2.630528,2.904057,3.055434,2.934302,3.247057,3.294245,3.206377,3.412208,3.177642
2019,2.691327,2.566346,2.937596,2.603904,2.500519,2.826615,2.999288,2.825962,3.251269,3.251269,3.106346,3.420462,3.055827
2020,2.258250,2.144019,2.483962,2.167519,2.073712,2.369846,2.583000,2.428500,2.809231,2.834115,2.711365,2.978096,2.550808
2021,2.420250,2.309750,2.637500,2.334250,2.243750,2.526750,2.719000,2.562500,2.950750,2.975250,2.848750,3.125000,2.680500


In [10]:
path = "complete_us_fuel_economy/"

all_files = glob.glob(path + "*.xlsx")

files_list = []

for filename in all_files:
    try:
        files_df =pd.read_excel(filename)
#     df = pd.read_excel(filename, index_col = None, header = 0)
        files_list.append(files_df)
    
    except: PermissionError

files_df = pd.concat(files_list, axis =0, ignore_index = True)

files_clean = files_df.loc[:,["Model Year",
                     "Mfr Name",
                     "Division",
                     "City FE (Guide) - Conventional Fuel",
                     "Hwy FE (Guide) - Conventional Fuel",
                     "Comb FE (Guide) - Conventional Fuel",
                     "City Unadj FE - Conventional Fuel",
                     "Hwy Unadj FE - Conventional Fuel",
                     "Comb Unadj FE - Conventional Fuel",
                     "City Unrd Adj FE - Conventional Fuel",
                     "Hwy Unrd Adj FE - Conventional Fuel",
                     "Comb Unrd Adj FE - Conventional Fuel",
                     "Fuel Usage Desc - Conventional Fuel"]]
files_clean

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,NaN,aston martin,Aston Martin Lagonda Ltd,14,20,16,16.9502,28.0198,20.6508,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
1,NaN,aston martin,Aston Martin Lagonda Ltd,12,19,14,15.0065,25.6094,18.4349,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
2,NaN,Audi,Audi,12,19,15,15.3,26.8,18.9614,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
3,NaN,Audi,Audi,13,18,15,15.4,25.0451,18.6283,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
4,NaN,Audi,Audi,13,20,16,15.8,24.8,18.8839,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14540,2022.0,Jaguar Land Rover L,Land Rover,17,22,19,20.84,30.5226,24.3104,16.7042,21.9137,18.7053,Gasoline (Premium Unleaded Recommended)
14541,2022.0,Jaguar Land Rover L,Land Rover,19,24,21,24.6746,35.1777,28.5044,18.5204,23.8248,20.5825,Gasoline (Premium Unleaded Recommended)
14542,2022.0,Jaguar Land Rover L,Land Rover,19,24,21,24.6746,35.1777,28.5044,18.5204,23.8248,20.5825,Gasoline (Premium Unleaded Recommended)
14543,2022.0,Jaguar Land Rover L,Land Rover,15,20,16,17.9015,27.0801,21.1233,14.5136,19.6238,16.4401,Gasoline (Premium Unleaded Recommended)


In [11]:
files_clean.dropna().groupby("Model Year").mean()

,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel
Model Year,,,,,,
2013.0,19.574871,26.977625,22.246988,19.536401,26.912754,22.239994
2014.0,19.855351,27.418060,22.597826,19.839688,27.408190,22.599637
2015.0,20.153477,27.924061,22.961631,20.132416,27.876310,22.954063
2016.0,20.430338,28.164880,23.258862,20.424168,28.132987,23.241010
2017.0,20.569936,27.795016,23.245981,20.569113,27.772557,23.231466
2018.0,20.651938,27.827132,23.311628,20.652907,27.809382,23.308930
2019.0,20.700787,27.645669,23.272441,20.681551,27.622681,23.266541
2020.0,20.675514,27.369007,23.175514,20.669748,27.370156,23.172568
2021.0,20.684943,27.033943,23.060052,20.693284,27.032448,23.065213


In [12]:
# Complete US Fuel Economy Data 2021
excel_file = "complete_us_fuel_economy/2021 FE Guide for DOE-release dates before 6-24-2021-no-sales -6-23-2021public.xlsx"
car_stats_2021 = pd.read_excel(excel_file)
car_stats_2021_clean = car_stats_2021.loc[:,["Model Year","Mfr Name","Division","City FE (Guide) - Conventional Fuel",
                                             "Hwy FE (Guide) - Conventional Fuel","Comb FE (Guide) - Conventional Fuel",
                                            "City Unadj FE - Conventional Fuel","Hwy Unadj FE - Conventional Fuel","Comb Unadj FE - Conventional Fuel",
                                            "City Unrd Adj FE - Conventional Fuel","Hwy Unrd Adj FE - Conventional Fuel","Comb Unrd Adj FE - Conventional Fuel",
                                            "Fuel Usage Desc - Conventional Fuel"]]
car_stats_2021_clean.head()

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,2021,Honda,Acura,21,22,21,28.7000,30.2000,29.3561,21.0000,21.7131,21.0000,Gasoline (Premium Unleaded Required)
1,2021,aston martin,Aston Martin Lagonda Ltd,14,21,17,17.3588,29.2584,21.2475,14.1001,21.0816,16.5693,Gasoline (Premium Unleaded Recommended)
2,2021,aston martin,Aston Martin Lagonda Ltd,18,24,20,22.5379,34.2945,26.6489,17.9972,24.4274,20.4155,Gasoline (Premium Unleaded Recommended)
3,2021,Volkswagen Group of,Audi,13,20,16,15.8765,26.0378,19.2586,13.3388,19.6687,15.5977,Gasoline (Premium Unleaded Required)
4,2021,Volkswagen Group of,Audi,14,23,17,15.8768,29.6317,20.0690,13.6958,22.9342,16.7281,Gasoline (Premium Unleaded Required)


In [13]:
# Complete US Fuel Economy Data 2020
excel_file = "complete_us_fuel_economy/2020 FE Guide-adds and corrections for DOE-OK for release-no-sales- 4-7-2021forpublic.xlsx"
car_stats_2020 = pd.read_excel(excel_file)
car_stats_2020_clean = car_stats_2020.loc[:,["Model Year","Mfr Name","Division","City FE (Guide) - Conventional Fuel",
                                             "Hwy FE (Guide) - Conventional Fuel","Comb FE (Guide) - Conventional Fuel",
                                            "City Unadj FE - Conventional Fuel","Hwy Unadj FE - Conventional Fuel","Comb Unadj FE - Conventional Fuel",
                                            "City Unrd Adj FE - Conventional Fuel","Hwy Unrd Adj FE - Conventional Fuel","Comb Unrd Adj FE - Conventional Fuel",
                                            "Fuel Usage Desc - Conventional Fuel"]]
car_stats_2020_clean.head()

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,2020,Honda,Acura,21,22,21,28.7000,30.2000,29.3561,21.0000,21.7131,21.0000,Gasoline (Premium Unleaded Required)
1,2020,FCA US LLC,ALFA ROMEO,24,34,28,28.7000,45.7000,34.4702,24.2492,34.3275,27.9406,Gasoline (Premium Unleaded Recommended)
2,2020,aston martin,Aston Martin Lagonda Ltd,14,21,17,17.3588,29.2584,21.2475,14.1001,21.0816,16.5693,Gasoline (Premium Unleaded Recommended)
3,2020,aston martin,Aston Martin Lagonda Ltd,18,24,20,22.5379,34.2945,26.6489,17.9972,24.4274,20.4155,Gasoline (Premium Unleaded Recommended)
4,2020,Volkswagen Group of,Audi,13,20,16,16.0159,26.8762,19.5755,13.4690,20.4562,15.9153,Gasoline (Premium Unleaded Required)


In [14]:
# Complete US Fuel Economy Data 2019
excel_file = "complete_us_fuel_economy/2019 FE Guide-for DOE-release dates before 12-19-2019-no-sales- 12_17_2019McLarenpublic.xlsx"
car_stats_2019 = pd.read_excel(excel_file)
car_stats_2019_clean = car_stats_2019.loc[:,["Model Year","Mfr Name","Division","City FE (Guide) - Conventional Fuel",
                                             "Hwy FE (Guide) - Conventional Fuel","Comb FE (Guide) - Conventional Fuel",
                                            "City Unadj FE - Conventional Fuel","Hwy Unadj FE - Conventional Fuel","Comb Unadj FE - Conventional Fuel",
                                            "City Unrd Adj FE - Conventional Fuel","Hwy Unrd Adj FE - Conventional Fuel","Comb Unrd Adj FE - Conventional Fuel",
                                            "Fuel Usage Desc - Conventional Fuel"]]
car_stats_2019_clean.head()

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,2019,Honda,Acura,21,22,21,28.7000,30.2000,29.3561,21.0000,21.7131,21.0000,Gasoline (Premium Unleaded Required)
1,2019,FCA US LLC,ALFA ROMEO,24,34,28,28.7000,45.7000,34.4702,24.2492,34.3275,27.9406,Gasoline (Premium Unleaded Recommended)
2,2019,aston martin,Aston Martin Lagonda Ltd,18,25,21,23.1719,35.2592,27.3986,18.4652,25.0596,20.9455,Gasoline (Premium Unleaded Recommended)
3,2019,Volkswagen Group of,Audi,23,31,26,28.3069,42.4085,33.2879,22.9796,30.9611,25.9952,Gasoline (Regular Unleaded Recommended)
4,2019,BMW,BMW,25,32,28,32.2000,45.9000,37.1959,24.9259,31.8536,27.6300,Gasoline (Premium Unleaded Recommended)


In [15]:
# Complete US Fuel Economy Data 2018
excel_file = "complete_us_fuel_economy/2018 FE Guide for DOE3 -all rel dates-no-sales-3-25-2019McLarenforpublic.xlsx"
car_stats_2018 = pd.read_excel(excel_file)
car_stats_2018_clean = car_stats_2018.loc[:,["Model Year","Mfr Name","Division","City FE (Guide) - Conventional Fuel",
                                             "Hwy FE (Guide) - Conventional Fuel","Comb FE (Guide) - Conventional Fuel",
                                            "City Unadj FE - Conventional Fuel","Hwy Unadj FE - Conventional Fuel","Comb Unadj FE - Conventional Fuel",
                                            "City Unrd Adj FE - Conventional Fuel","Hwy Unrd Adj FE - Conventional Fuel","Comb Unrd Adj FE - Conventional Fuel",
                                            "Fuel Usage Desc - Conventional Fuel"]]
car_stats_2018_clean.head()

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,2018,aston martin,Aston Martin Lagonda Ltd,15,21,17,18.8306,30.8924,22.8444,15.1397,20.5453,17.1729,Gasoline (Premium Unleaded Recommended)
1,2018,aston martin,Aston Martin Lagonda Ltd,18,24,20,22.7207,33.8295,26.6603,18.1323,24.1217,20.4132,Gasoline (Premium Unleaded Recommended)
2,2018,aston martin,Aston Martin Lagonda Ltd,13,21,16,16.5327,28.7057,20.4316,13.4660,20.7097,15.9815,Gasoline (Premium Unleaded Recommended)
3,2018,aston martin,Aston Martin Lagonda Ltd,14,21,17,17.2984,29.8618,21.3382,14.0538,21.4866,16.6449,Gasoline (Premium Unleaded Recommended)
4,2018,BMW,Mini,24,32,27,31.3604,46.5690,36.7632,24.3407,32.2701,27.3667,Gasoline (Premium Unleaded Recommended)


In [16]:
# Complete US Fuel Economy Data 2017
excel_file = "complete_us_fuel_economy/2017 FE Guide for DOE-release dates before 3-25-2019-no sales-9-19-2019McLarenforpublic.xlsx"
car_stats_2017 = pd.read_excel(excel_file)
car_stats_2017_clean = car_stats_2017.loc[:,["Model Year","Mfr Name","Division","City FE (Guide) - Conventional Fuel",
                                             "Hwy FE (Guide) - Conventional Fuel","Comb FE (Guide) - Conventional Fuel",
                                            "City Unadj FE - Conventional Fuel","Hwy Unadj FE - Conventional Fuel","Comb Unadj FE - Conventional Fuel",
                                            "City Unrd Adj FE - Conventional Fuel","Hwy Unrd Adj FE - Conventional Fuel","Comb Unrd Adj FE - Conventional Fuel",
                                            "Fuel Usage Desc - Conventional Fuel"]]
car_stats_2017_clean.head()

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,2017,Honda,Acura,21,22,21,28.7000,30.2000,29.3561,21.0000,21.7131,21.0000,Gasoline (Premium Unleaded Required)
1,2017,FCA US LLC,ALFA ROMEO,24,34,28,28.7000,45.7000,34.4702,24.2492,34.3275,27.9406,Gasoline (Premium Unleaded Recommended)
2,2017,aston martin,Aston Martin Lagonda Ltd,12,18,14,14.7893,25.3011,18.1901,12.1164,18.3976,14.3158,Gasoline (Premium Unleaded Recommended)
3,2017,aston martin,Aston Martin Lagonda Ltd,10,16,12,12.2894,22.3680,15.4150,10.1534,16.3763,12.2477,Gasoline (Premium Unleaded Recommended)
4,2017,Volkswagen Group of,Audi,14,22,17,17.2263,29.0421,21.0869,14.4642,21.6225,16.9962,Gasoline (Premium Unleaded Recommended)


In [17]:
#let's merge all dataframes together from 2017-2021


frames = [car_stats_2017_clean, car_stats_2018_clean, car_stats_2019_clean,
         car_stats_2020_clean, car_stats_2021_clean]

car_mpg = pd.concat(frames)
car_mpg.count()

# make sure merge is doing what we think it is by looking at counts
# result.count()
# car_stats_2017_clean.count()
# car_stats_2018_clean.count()

Model Year                              6121
Mfr Name                                6121
Division                                6121
City FE (Guide) - Conventional Fuel     6121
Hwy FE (Guide) - Conventional Fuel      6121
Comb FE (Guide) - Conventional Fuel     6121
City Unadj FE - Conventional Fuel       6121
Hwy Unadj FE - Conventional Fuel        6121
Comb Unadj FE - Conventional Fuel       6121
City Unrd Adj FE - Conventional Fuel    6121
Hwy Unrd Adj FE - Conventional Fuel     6121
Comb Unrd Adj FE - Conventional Fuel    6121
Fuel Usage Desc - Conventional Fuel     6121
dtype: int64

In [18]:
#rename `Model Year` column to make sql easier
car_mpg = car_mpg.rename(columns={"Model Year": "Year"})
car_mpg.head()

,Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,2017,Honda,Acura,21,22,21,28.7000,30.2000,29.3561,21.0000,21.7131,21.0000,Gasoline (Premium Unleaded Required)
1,2017,FCA US LLC,ALFA ROMEO,24,34,28,28.7000,45.7000,34.4702,24.2492,34.3275,27.9406,Gasoline (Premium Unleaded Recommended)
2,2017,aston martin,Aston Martin Lagonda Ltd,12,18,14,14.7893,25.3011,18.1901,12.1164,18.3976,14.3158,Gasoline (Premium Unleaded Recommended)
3,2017,aston martin,Aston Martin Lagonda Ltd,10,16,12,12.2894,22.3680,15.4150,10.1534,16.3763,12.2477,Gasoline (Premium Unleaded Recommended)
4,2017,Volkswagen Group of,Audi,14,22,17,17.2263,29.0421,21.0869,14.4642,21.6225,16.9962,Gasoline (Premium Unleaded Recommended)


In [19]:
#we can do a group by to get averages for all cars in dataframe
avg_mpg = car_mpg.groupby(['Year']).mean()
avg_mpg.T

Year,2017,2018,2019,2020,2021
City FE (Guide) - Conventional Fuel,20.569936,20.651938,20.700787,20.675514,20.684943
Hwy FE (Guide) - Conventional Fuel,27.795016,27.827132,27.645669,27.369007,27.033943
Comb FE (Guide) - Conventional Fuel,23.245981,23.311628,23.272441,23.175514,23.060052
City Unadj FE - Conventional Fuel,26.289742,26.385963,26.456764,26.459292,26.545104
Hwy Unadj FE - Conventional Fuel,39.681418,39.770338,39.591396,39.177124,38.847437
Comb Unadj FE - Conventional Fuel,30.894665,31.001437,30.991457,30.866840,30.829873
City Unrd Adj FE - Conventional Fuel,20.569113,20.652907,20.681551,20.669748,20.693284
Hwy Unrd Adj FE - Conventional Fuel,27.772557,27.809382,27.622681,27.370156,27.032448
Comb Unrd Adj FE - Conventional Fuel,23.231466,23.308930,23.266541,23.172568,23.065213


In [20]:
#for our convenience, the gas prices table of last 5 years are here so there's less scrolling. 
last_5yrs.T

Year,2017,2018,2019,2020,2021
All Grades All Formulations,2.527885,2.813189,2.691327,2.258250,2.42025
All Grades Conventional,2.447769,2.714358,2.566346,2.144019,2.30975
All Grades Reformulated,2.691231,3.008151,2.937596,2.483962,2.63750
Regular All Formulations,2.414615,2.718547,2.603904,2.167519,2.33425
Regular Conventional,2.333173,2.630528,2.500519,2.073712,2.24375
Regular Reformulated,2.585500,2.904057,2.826615,2.369846,2.52675
Midgrade All Formulations,2.681615,3.055434,2.999288,2.583000,2.71900
Midgrade Conventional,2.595096,2.934302,2.825962,2.428500,2.56250
Midgrade Reformulated,2.848885,3.247057,3.251269,2.809231,2.95075
Premium All Formulations,2.916500,3.294245,3.251269,2.834115,2.97525


In [21]:
#prepare connection info for postgres
#make sure creds match, esp database name
protocol = 'postgresql'
username = 'postgres'
password = 'admin'
host = 'localhost'
port = 5432
database_name = 'mydb'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [22]:
engine.table_names()

C:\Users\yhuan\AppData\Local\Temp/ipykernel_27480/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['people', 'fauna_vertabrate', 'us_gas_price', 'car_mpg']

In [23]:
#write us gas price df to postgres
last_5yrs.to_sql(name='us_gas_price', con=engine, if_exists='replace', index=True)

In [24]:
#write car mpg df to postgres
avg_mpg.to_sql(name='car_mpg', con=engine, if_exists='replace', index=True)

In [25]:
pd.read_sql_query('select * from us_gas_price', con=engine).head()

,Year,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
0,2017,2.527885,2.447769,2.691231,2.414615,2.333173,2.585500,2.681615,2.595096,2.848885,2.916500,2.864577,3.013019,2.650038
1,2018,2.813189,2.714358,3.008151,2.718547,2.630528,2.904057,3.055434,2.934302,3.247057,3.294245,3.206377,3.412208,3.177642
2,2019,2.691327,2.566346,2.937596,2.603904,2.500519,2.826615,2.999288,2.825962,3.251269,3.251269,3.106346,3.420462,3.055827
3,2020,2.258250,2.144019,2.483962,2.167519,2.073712,2.369846,2.583000,2.428500,2.809231,2.834115,2.711365,2.978096,2.550808
4,2021,2.420250,2.309750,2.637500,2.334250,2.243750,2.526750,2.719000,2.562500,2.950750,2.975250,2.848750,3.125000,2.680500


In [26]:
pd.read_sql_query('select * from car_mpg', con=engine).head()

,Year,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel
0,2017,20.569936,27.795016,23.245981,26.289742,39.681418,30.894665,20.569113,27.772557,23.231466
1,2018,20.651938,27.827132,23.311628,26.385963,39.770338,31.001437,20.652907,27.809382,23.308930
2,2019,20.700787,27.645669,23.272441,26.456764,39.591396,30.991457,20.681551,27.622681,23.266541
3,2020,20.675514,27.369007,23.175514,26.459292,39.177124,30.866840,20.669748,27.370156,23.172568
4,2021,20.684943,27.033943,23.060052,26.545104,38.847437,30.829873,20.693284,27.032448,23.065213
